### Predict Forest Classification on New Images

This notebook will load in the saved model that used transfer learning with MobileNetV2 wit the top 14 layers fine-tuned to the training images. Then you can predict the classification for an entire directory containing image files. The output is a list of tuples with each image's file name and it's predicted classification.

In [52]:
#import packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras

In [53]:
reconstructed_model = keras.models.load_model("Forest_MobileNetV2_Transfer_Model")

In [54]:
reconstructed_model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 64, 64, 3)]       0         
                                                                 
 tf.math.truediv (TFOpLambda  (None, 64, 64, 3)        0         
 )                                                               
                                                                 
 tf.math.subtract (TFOpLambd  (None, 64, 64, 3)        0         
 a)                                                              
                                                                 
 mobilenetv2_1.00_224 (Funct  (None, 2, 2, 1280)       2257984   
 ional)                                                          
                                                                 
 global_average_pooling2d (G  (None, 1280)             0         
 lobalAveragePooling2D)                                      

In [55]:
channels = 3 #RGB
img_w = 64
img_h = 64
input_shape = (img_h, img_w, channels)
directory = "../additional_images_for_testing/mixed_test"

In [57]:
import os

predicted_classes = []
image_files = []

for filename in os.listdir(directory):
    f = os.path.join(directory, filename)
    if os.path.isfile(f):
        test_img = keras.preprocessing.image.load_img(f, target_size=(img_w,img_h))

        img_array = keras.preprocessing.image.img_to_array(test_img)
        img_array = tf.expand_dims(img_array, axis=0)  # Create batch axis

        class_names = ['NoTrees', 'Trees']

        prediction = reconstructed_model.predict(img_array, verbose=0)
        prediction = tf.nn.sigmoid(prediction)
        prediction = tf.where(prediction < 0.5, 0, 1)
        
        #print(filename)
        image_files.append(filename)
        
        if prediction[0] == 0:
            #print(class_names[0])
            predicted_classes.append(class_names[0])
        else: 
            #print(class_names[1])
            predicted_classes.append(class_names[1])

merged_list = list(zip(image_files, predicted_classes))            
print(merged_list)        

[('desert_1.jpg', 'NoTrees'), ('desert_10.jpg', 'Trees'), ('desert_11.jpg', 'Trees'), ('desert_12.jpg', 'Trees'), ('desert_13.jpg', 'Trees'), ('desert_14.jpg', 'Trees'), ('desert_15.jpg', 'Trees'), ('desert_16.jpg', 'Trees'), ('desert_2.jpg', 'NoTrees'), ('desert_3.jpg', 'Trees'), ('desert_4.jpg', 'NoTrees'), ('desert_5.jpg', 'Trees'), ('desert_6.jpg', 'Trees'), ('desert_7.jpg', 'NoTrees'), ('desert_8.jpg', 'Trees'), ('desert_9.jpg', 'Trees'), ('Forest_2.jpg', 'Trees'), ('Forest_20.jpg', 'Trees'), ('Forest_21.jpg', 'NoTrees'), ('Forest_22.jpg', 'Trees'), ('Forest_23.jpg', 'Trees'), ('Forest_29.jpg', 'Trees'), ('Forest_3.jpg', 'Trees'), ('Forest_30.jpg', 'Trees'), ('Forest_31.jpg', 'Trees'), ('Forest_32.jpg', 'Trees'), ('Forest_4.jpg', 'Trees'), ('Forest_5.jpg', 'Trees'), ('Forest_6.jpg', 'Trees'), ('Forest_7.jpg', 'Trees'), ('Forest_8.jpg', 'Trees'), ('Forest_9.jpg', 'Trees'), ('SeaLake_1.jpg', 'NoTrees'), ('SeaLake_11.jpg', 'NoTrees'), ('SeaLake_16.jpg', 'NoTrees'), ('SeaLake_18.jpg',

These are from a different set of satellite images, and the model does a good job on the Forest images and on the SeaLake images, but has a little trouble with some of the Desert images, as there were none like this in the training data. I threw those in there to see how it would do. It is not likely that one would needing to classify whether desert images are forested or not, so it's not a huge problem that the model does not work well here.